<a href="https://colab.research.google.com/github/julianencisoizquierdo/NLP_Python_Airline_Customer_Service/blob/main/NLP_Case02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# suppress warnings
import warnings;
warnings.filterwarnings('ignore');

# common imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import html
import math
import re
import glob
import os
import sys
import json
import random
import pprint as pp
import textwrap
import sqlite3
import logging
from fractions import Fraction

import spacy
import nltk

import seaborn as sns
sns.set_style("darkgrid")

from tqdm.auto import tqdm
# register `pandas.progress_apply` and `pandas.Series.map_apply` with `tqdm`
tqdm.pandas()


# Machine learning packages
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay, f1_score, cohen_kappa_score, roc_auc_score, average_precision_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder


# pandas display options
# https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html#available-options
pd.options.display.max_columns = 30 # default 20
pd.options.display.max_rows = 60 # default 60
pd.options.display.float_format = '{:.2f}'.format
# pd.options.display.precision = 2
pd.options.display.max_colwidth = 200 # default 50; -1 = all
# otherwise text between $ signs will be interpreted as formula and printed in italic
pd.set_option('display.html.use_mathjax', False)

# np.set_printoptions(edgeitems=3) # default 3

plot_params = {'figure.figsize': (8, 4),
               'axes.labelsize': 'large',
               'axes.titlesize': 'large',
               'xtick.labelsize': 'large',
               'ytick.labelsize':'large',
               'figure.dpi': 100}




In [ ]:
# Julie: I had to change this to my drive, as I couldnt access the shared folder from Colab
BASE_DIR = '/content/drive/MyDrive/NLP Group Project/Datasets/packages'
os.chdir(BASE_DIR)

# **Case II**

##**Step 1: Recode the outcome into a binary variable**

In [ ]:
#Jade: I created another df_java for me to run the code, using the shared drive as I couldn't read it
df_java = pd.read_csv("/content/drive/MyDrive/NLP Group Project/Datasets/eclipse_jdt.csv")

In [ ]:
df_java.head(3)

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time
0,1518,P1,Debug,NaN,Icons needed for actions (1GI5UXW),JGS (8/8/01 5:20:19 PM); \tWe need enabled; disabled & hover icons for the following actions:; \t\tCopyToClipboardActionDelegate; \t\tRelaunchActionDelegate; \t\tTerminateAndRemoveActionDelegate; ...,VERIFIED,FIXED,2.0,2001-10-10 22:14:00 -0400,2001-10-18 11:51:14 -0400
1,1519,P3,Debug,NaN,README: Hit count not reset (1GET20Y),JGS (6/4/01 11:43:47 AM); \tSet a breakpoint in InfiniteLoop with a hit count. When suspended;; \tright click breakpoint in Breakpoints viewer and enable. Hit count; \tstays at previous value. ...,RESOLVED,WORKSFORME,2.0,2001-10-10 22:14:00 -0400,2001-11-28 13:42:46 -0500
2,1520,P3,Debug,NaN,Use styled text in console (1G9S1YF),DW (2/26/01 5:56:36 PM); \tCould allow users to use styled text in console; with preference support. This would; \tallow console input to be shown in bold; etc.,CLOSED,WONTFIX,2.0,2001-10-10 22:14:00 -0400,2002-06-26 11:32:05 -0400


In [ ]:
df_java.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45296 entries, 0 to 45295
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Issue_id          45296 non-null  int64  
 1   Priority          45296 non-null  object 
 2   Component         45296 non-null  object 
 3   Duplicated_issue  7688 non-null   float64
 4   Title             45296 non-null  object 
 5   Description       45253 non-null  object 
 6   Status            45296 non-null  object 
 7   Resolution        45296 non-null  object 
 8   Version           45296 non-null  object 
 9   Created_time      45296 non-null  object 
 10  Resolved_time     45296 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 3.8+ MB


In [ ]:
df_java = df_java.drop(columns = ["Duplicated_issue"])
df_java.head(3)

,Issue_id,Priority,Component,Title,Description,Status,Resolution,Version,Created_time,Resolved_time
0,1518,else,Debug,Icons needed for actions (1GI5UXW),JGS (8/8/01 5:20:19 PM); \tWe need enabled; disabled & hover icons for the following actions:; \t\tCopyToClipboardActionDelegate; \t\tRelaunchActionDelegate; \t\tTerminateAndRemoveActionDelegate; ...,VERIFIED,FIXED,2.0,2001-10-10 22:14:00 -0400,2001-10-18 11:51:14 -0400
1,1519,P3,Debug,README: Hit count not reset (1GET20Y),JGS (6/4/01 11:43:47 AM); \tSet a breakpoint in InfiniteLoop with a hit count. When suspended;; \tright click breakpoint in Breakpoints viewer and enable. Hit count; \tstays at previous value. ...,RESOLVED,WORKSFORME,2.0,2001-10-10 22:14:00 -0400,2001-11-28 13:42:46 -0500
2,1520,P3,Debug,Use styled text in console (1G9S1YF),DW (2/26/01 5:56:36 PM); \tCould allow users to use styled text in console; with preference support. This would; \tallow console input to be shown in bold; etc.,CLOSED,WONTFIX,2.0,2001-10-10 22:14:00 -0400,2002-06-26 11:32:05 -0400


In [ ]:
# Change priority variable into a binary variable 'P3' vs 'else'
df_java["Priority"] = df_java["Priority"].apply(lambda x: "P3" if x == "P3" else "else")
df_java.head(3)

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time
0,1518,else,Debug,NaN,Icons needed for actions (1GI5UXW),JGS (8/8/01 5:20:19 PM); \tWe need enabled; disabled & hover icons for the following actions:; \t\tCopyToClipboardActionDelegate; \t\tRelaunchActionDelegate; \t\tTerminateAndRemoveActionDelegate; ...,VERIFIED,FIXED,2.0,2001-10-10 22:14:00 -0400,2001-10-18 11:51:14 -0400
1,1519,P3,Debug,NaN,README: Hit count not reset (1GET20Y),JGS (6/4/01 11:43:47 AM); \tSet a breakpoint in InfiniteLoop with a hit count. When suspended;; \tright click breakpoint in Breakpoints viewer and enable. Hit count; \tstays at previous value. ...,RESOLVED,WORKSFORME,2.0,2001-10-10 22:14:00 -0400,2001-11-28 13:42:46 -0500
2,1520,P3,Debug,NaN,Use styled text in console (1G9S1YF),DW (2/26/01 5:56:36 PM); \tCould allow users to use styled text in console; with preference support. This would; \tallow console input to be shown in bold; etc.,CLOSED,WONTFIX,2.0,2001-10-10 22:14:00 -0400,2002-06-26 11:32:05 -0400


In [ ]:
df_java.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45296 entries, 0 to 45295
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Issue_id       45296 non-null  int64 
 1   Priority       45296 non-null  object
 2   Component      45296 non-null  object
 3   Title          45296 non-null  object
 4   Description    45253 non-null  object
 5   Status         45296 non-null  object
 6   Resolution     45296 non-null  object
 7   Version        45296 non-null  object
 8   Created_time   45296 non-null  object
 9   Resolved_time  45296 non-null  object
dtypes: int64(1), object(9)
memory usage: 3.5+ MB


In [ ]:
# The dataset is highly imbalanced with 88% of priority level P3
df_java["Priority"].value_counts(normalize = True)

P3     0.88
else   0.12
Name: Priority, dtype: float64

In [ ]:
# Concatenate Title and Description into Text column, and only keep Text and Priority columns
java = df_java[["Title", "Description", "Priority"]]
java = java.dropna()
java["Text"] = df_java["Title"] + ' ' + java["Description"]
java = java.drop(columns = ["Title", "Description"])
java.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45253 entries, 0 to 45295
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Priority  45253 non-null  object
 1   Text      45253 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


In [ ]:
java.head(3)

,Priority,Text
0,else,Icons needed for actions (1GI5UXW) JGS (8/8/01 5:20:19 PM); \tWe need enabled; disabled & hover icons for the following actions:; \t\tCopyToClipboardActionDelegate; \t\tRelaunchActionDelegate; \t\...
1,P3,README: Hit count not reset (1GET20Y) JGS (6/4/01 11:43:47 AM); \tSet a breakpoint in InfiniteLoop with a hit count. When suspended;; \tright click breakpoint in Breakpoints viewer and enable. H...
2,P3,Use styled text in console (1G9S1YF) DW (2/26/01 5:56:36 PM); \tCould allow users to use styled text in console; with preference support. This would; \tallow console input to be shown in bold; etc.


In [ ]:
# Clean java with the function from the preparation module and select only clean text with more than 20 characters

from packages.preparation import clean

java["Text"] = java["Text"].apply(clean)
java = java[java["Text"].str.len() > 20]
java.head(3)

ModuleNotFoundError: No module named 'packages.preparation'

In [ ]:
# Split dataset into test and training samples
java_X_train, java_X_test, java_Y_train, java_Y_test = train_test_split(java["Text"],
                                                                        java["Priority"],
                                                                        test_size = 0.2, # We use 80% as training and 20% as test sets
                                                                        random_state = 1,
                                                                        stratify = java["Priority"]) # We stratify to keep the original class distribution in the training and test samples
print("Train data:", java_X_train.shape[0])
print("Test data:", java_X_test.shape[0])

Train data: 36178
Test data: 9045


##**Step 2: Vectorization**

In [ ]:
# Vectorize java_X_train as a bag of words with TFIDF, keep tokens with document frequency of 10+, include bigrams, use english stopwords
# Use TFIDF to take into account word frequency in each text and in the entire dataset
tfidf = TfidfVectorizer(min_df = 10, ngram_range = (1, 2), stop_words = "english")
java_X_train_tf = tfidf.fit_transform(java_X_train)

##**Step 3: Build reference results**

In [ ]:
# Also vectorize the test set
java_X_test_tf = tfidf.transform(java_X_test)

In [ ]:
# Trivial classifier (always predicts the majority class)
trivial = DummyClassifier(strategy = "most_frequent", random_state = 1)
trivial.fit(java_X_train_tf, java_Y_train)
Y_pred_baseline = trivial.predict(java_X_test)

In [ ]:
# K-Nearest Neighbor
%%time
knn = KNeighborsClassifier(n_neighbors = 5, metric = "euclidean")
knn.fit(java_X_train_tf, java_Y_train)
y_pred_knn = knn.predict(java_X_test_tf)

CPU times: user 28min 33s, sys: 1.6 s, total: 28min 35s
Wall time: 28min 36s


In [ ]:
# Support Vector Machine
%%time
svm = SVC()
svm.fit(java_X_train_tf, java_Y_train)
y_pred_svm = svm.predict(java_X_test_tf)

CPU times: user 22min 34s, sys: 1.76 s, total: 22min 36s
Wall time: 22min 36s


In [ ]:
# Logistic Regression
%%time
logistic_regression = LogisticRegression(n_jobs = -1)
logistic_regression.fit(java_X_train_tf, java_Y_train)
y_pred_logistic_regression = logistic_regression.predict(java_X_test_tf)

CPU times: user 132 ms, sys: 118 ms, total: 249 ms
Wall time: 3.09 s


In [ ]:
# Random Forest
%%time
random_forest = RandomForestClassifier(random_state = 1, n_jobs = -1)
random_forest.fit(java_X_train_tf, java_Y_train)
y_pred_random_forest = random_forest.predict(java_X_test_tf)

CPU times: user 1min 51s, sys: 403 ms, total: 1min 51s
Wall time: 1min 3s


In [ ]:
# Gradient Boosting Classifier
%%time
gbm = GradientBoostingClassifier()
gbm.fit(java_X_train_tf, java_Y_train)
y_pred_gbm = gbm.predict(java_X_test_tf)

CPU times: user 1min 34s, sys: 187 ms, total: 1min 35s
Wall time: 1min 35s


In [ ]:
# Accuracy
print("Accuracy Trivial Model: ", accuracy_score(java_Y_test ,Y_pred_baseline))
print("Accuracy KNN: ", accuracy_score(java_Y_test ,y_pred_knn))
print("Accuracy SVM: ", accuracy_score(java_Y_test ,y_pred_svm))
print("Accuracy LogisticRegression: ", accuracy_score(java_Y_test ,y_pred_logistic_regression))
print("Accuracy Random Forest: ", accuracy_score(java_Y_test ,y_pred_random_forest))
print("Accuracy GBM: ", accuracy_score(java_Y_test, y_pred_gbm))

Accuracy Trivial Model:  0.8766169154228856
Accuracy SVM:  0.8798231066887783
Accuracy LogisticRegression:  0.8793808734107241
Accuracy Random Forest:  0.8805970149253731
Accuracy KNN:  0.8770591487009397
Accuracy GBM:  0.8821448313985627


In [ ]:
# F1 score
print("F1-score Trivial Model: ", f1_score(java_Y_test ,Y_pred_baseline, pos_label = "P3"))
print("F1-score KNN: ", f1_score(java_Y_test ,y_pred_knn, pos_label = "P3"))
print("F1-score SVM: ", f1_score(java_Y_test ,y_pred_svm, pos_label = "P3"))
print("F1-score LogisticRegression: ", f1_score(java_Y_test ,y_pred_logistic_regression, pos_label = "P3"))
print("F1-score Random Forest: ", f1_score(java_Y_test ,y_pred_random_forest, pos_label = "P3"))
print("F1-score GBM: ", f1_score(java_Y_test ,y_pred_gbm, pos_label = "P3"))

F1-score Trivial Model:  0.9342523860021209
F1-score SVM:  0.9356614382953538
F1-score LogisticRegression:  0.935325152646867
F1-score Random Forest:  0.9360644091877812
F1-score KNN:  0.9341232227488151
F1-score GBM:  0.936780927529356


In [ ]:
# Cohen's Kappa
print("Cohen's kappa Trivial Model: ", cohen_kappa_score(java_Y_test ,Y_pred_baseline))
print("Cohen's kappa KNN: ", cohen_kappa_score(java_Y_test ,y_pred_knn))
print("Cohen's kappa SVM: ", cohen_kappa_score(java_Y_test ,y_pred_svm))
print("Cohen's kappa LogisticRegression: ", cohen_kappa_score(java_Y_test ,y_pred_logistic_regression))
print("Cohen's kappa Random Forest: ", cohen_kappa_score(java_Y_test ,y_pred_random_forest))
print("Cohen's kappa GBM: ", cohen_kappa_score(java_Y_test ,y_pred_gbm))

Cohen's kappa Trivial Model:  0.0
Cohen's kappa SVM:  0.0752914169240626
Cohen's kappa LogisticRegression:  0.08709770079054835
Cohen's kappa Random Forest:  0.08300905501100841
Cohen's kappa KNN:  0.06302976848226516
Cohen's kappa GBM:  0.11193470627439162


In [ ]:
# ROC AUC
label_encoder = LabelEncoder()
encoded_java_Y_test = label_encoder.fit_transform(java_Y_test)
encoded_Y_pred_baseline = label_encoder.fit_transform(Y_pred_baseline)
encoded_Y_pred_svm = label_encoder.fit_transform(y_pred_svm)
encoded_Y_pred_logistic_regression = label_encoder.fit_transform(y_pred_logistic_regression)
encoded_Y_pred_random_forest = label_encoder.fit_transform(y_pred_random_forest)
encoded_Y_pred_knn = label_encoder.fit_transform(y_pred_knn)
encoded_Y_pred_gbm = label_encoder.fit_transform(y_pred_gbm)
print("ROC AUC Trivial Model: ", roc_auc_score(encoded_java_Y_test, encoded_Y_pred_baseline))
print("ROC AUC KNN: ", roc_auc_score(encoded_java_Y_test ,encoded_Y_pred_knn))
print("ROC AUC SVM: ", roc_auc_score(encoded_java_Y_test ,encoded_Y_pred_svm))
print("ROC AUC LogisticRegression: ", roc_auc_score(encoded_java_Y_test ,encoded_Y_pred_logistic_regression))
print("ROC AUC Random Forest: ", roc_auc_score(encoded_java_Y_test ,encoded_Y_pred_random_forest))
print("ROC AUC GBM: ", roc_auc_score(encoded_java_Y_test ,encoded_Y_pred_gbm))

ROC AUC Trivial Model:  0.5
ROC AUC SVM:  0.5226170570262695
ROC AUC LogisticRegression:  0.5265994776219594
ROC AUC Random Forest:  0.5249833197043112
ROC AUC KNN:  0.5191157205684319
ROC AUC GBM:  0.5343354732932192


In [ ]:
# PRC AUC
print("PRC AUC Trivial Model: ", average_precision_score(encoded_java_Y_test, encoded_Y_pred_baseline))
print("PRC AUC KNN: ", average_precision_score(encoded_java_Y_test ,encoded_Y_pred_knn))
print("PRC AUC SVM: ", average_precision_score(encoded_java_Y_test ,encoded_Y_pred_svm))
print("PRC AUC LogisticRegression: ", average_precision_score(encoded_java_Y_test ,encoded_Y_pred_logistic_regression))
print("PRC AUC Random Forest: ", average_precision_score(encoded_java_Y_test ,encoded_Y_pred_random_forest))
print("PRC AUC GBM: ", average_precision_score(encoded_java_Y_test ,encoded_Y_pred_gbm))

PRC AUC Trivial Model:  0.12338308457711443
PRC AUC SVM:  0.15048765970067995
PRC AUC LogisticRegression:  0.15225243470308467
PRC AUC Random Forest:  0.15489887088728052
PRC AUC KNN:  0.14085331938644308
PRC AUC GBM:  0.166919220143522


In [ ]:
# Define a H-Measure function
def h_measure(y_true, y_pred):
  "Calculate the H-measure given the predicted Y and the true value of Y"
  precision = precision_score(y_true, y_pred)
  recall = recall_score (y_true, y_pred)
  h = 2 * (precision * recall) / (precision + recall)
  return h

In [ ]:
# H-measure
print("H-measure Trivial Model: ", h_measure(encoded_java_Y_test, encoded_Y_pred_baseline))
print("H-measure KNN: ", h_measure(encoded_java_Y_test ,encoded_Y_pred_knn))
print("H-measure SVM: ", h_measure(encoded_java_Y_test ,encoded_Y_pred_svm))
print("H-measure LogisticRegression: ", h_measure(encoded_java_Y_test ,encoded_Y_pred_logistic_regression))
print("H-measure Random Forest: ", h_measure(encoded_java_Y_test ,encoded_Y_pred_random_forest))
print("PH-measure GBM: ", h_measure(encoded_java_Y_test ,encoded_Y_pred_gbm))

NameError: name 'encoded_java_Y_test' is not defined

##**Step 4: Undersampling the majority class**

In [ ]:
# We create 2 dataframes, one with only P3 observations and another one with only 'else' observations
majority = java[java['Priority'] == 'P3']
minority = java[java['Priority'] != 'P3']

In [ ]:
# Use resample function to undersample the majority class
from sklearn.utils import resample
undersampled_majority = resample(
    majority,
    replace = False, # We want to sample without replacement
    n_samples = len(minority), # We match the minority class size
    random_state = 1
)

In [ ]:
# Combine the undersampled majority class with the entire minority class
undersampled_java = pd.concat([undersampled_majority, minority])

In [ ]:
undersampled_java.head()

,Priority,Text
32335,P3,TVT3.2:TCT913: tw Wrong shortcut for Content Assist for T.Chinese OS RHEL 3.1; Must fix or not a must fix: NO; Build date: 0611; Component name = BASE; Blocking: NO; Language: tw; Bitmap Location:...
4049,P3,Template for null is incorrectly displayed in codeassist popup Build 20020305; ; When selecting the null template; it displays ; incorrectly null; and then inserts null .
22930,P3,Improper behavior when trying to indent javadoc line Consider this code (suppose youre fixing javadocs):; ; public class Foo; {; /**; * something; void foo(); {}; }; ; Place cursor at the start of...
20145,P3,Externalize Strings at end of file doesnt break lines correctly I2004-06-10; ; I externalized a String such that the new key in the properties file was; inserted at the end of file. I ended up wit...
15826,P3,Jardesc should be able to store destination location relative to the workspace One of the things Im doing is generating a jar to put into another project. ; Id like to share that jar description w...


In [ ]:
undersampled_java['Priority'].value_counts(normalize = True)

P3     0.50
else   0.50
Name: Priority, dtype: float64

In [ ]:
# Split the undersampled dataset into the training and test sets
undersampled_java_X_train, undersampled_java_X_test, undersampled_java_Y_train, undersampled_java_Y_test = train_test_split(undersampled_java["Text"],
                                                                        undersampled_java["Priority"],
                                                                        test_size = 0.2, # We still use 80% as training and 20% as test sets
                                                                        random_state = 1,
                                                                        stratify = undersampled_java["Priority"]) # We stratify to keep the balance in classes between the train and test sets
print("Train data:", undersampled_java_X_train.shape[0])
print("Test data:", undersampled_java_X_test.shape[0])

Train data: 8926
Test data: 2232


In [ ]:
# Vectorize the train and test data
undersampled_java_X_train_tf = tfidf.fit_transform(undersampled_java_X_train)
undersampled_java_X_test_tf = tfidf.transform(undersampled_java_X_test)

In [ ]:
# Fit a new Trivial classifier
undersampled_trivial = DummyClassifier(strategy = "most_frequent", random_state = 1)
undersampled_trivial.fit(undersampled_java_X_train_tf, undersampled_java_Y_train)
undersampled_Y_pred_baseline = undersampled_trivial.predict(undersampled_java_X_test_tf)

In [ ]:
# K-Nearest Neighbor
%%time
undersampled_knn = KNeighborsClassifier(n_neighbors = 5, metric = "euclidean")
undersampled_knn.fit(undersampled_java_X_train_tf, undersampled_java_Y_train)
undersampled_Y_pred_knn = undersampled_knn.predict(undersampled_java_X_test_tf)

CPU times: user 1min 38s, sys: 193 ms, total: 1min 38s
Wall time: 1min 51s


In [ ]:
# Support Vector Machine
%%time
undersampled_svm = SVC()
undersampled_svm.fit(undersampled_java_X_train_tf, undersampled_java_Y_train)
undersampled_Y_pred_svm = undersampled_svm.predict(undersampled_java_X_test_tf)

CPU times: user 44.2 s, sys: 120 ms, total: 44.3 s
Wall time: 44.4 s


In [ ]:
# Logistic Regression
%%time
undersampled_logistic_regression = LogisticRegression(n_jobs = -1)
undersampled_logistic_regression.fit(undersampled_java_X_train_tf, undersampled_java_Y_train)
undersampled_Y_pred_logistic_regression = undersampled_logistic_regression.predict(undersampled_java_X_test_tf)

CPU times: user 41.8 ms, sys: 68.9 ms, total: 111 ms
Wall time: 1.43 s


In [ ]:
# Random Forest
%%time
undersampled_random_forest = RandomForestClassifier(random_state = 1, n_jobs = -1)
undersampled_random_forest.fit(undersampled_java_X_train_tf, undersampled_java_Y_train)
undersampled_Y_pred_random_forest = undersampled_random_forest.predict(undersampled_java_X_test_tf)

CPU times: user 14.8 s, sys: 81.5 ms, total: 14.9 s
Wall time: 15.8 s


In [ ]:
# GradientBoostingClassifier
%%time
undersampled_gbm = GradientBoostingClassifier()
undersampled_gbm.fit(undersampled_java_X_train_tf, undersampled_java_Y_train)
undersampled_Y_pred_gbm = undersampled_gbm.predict(undersampled_java_X_test_tf)

CPU times: user 19.2 s, sys: 48.7 ms, total: 19.2 s
Wall time: 22.6 s


In [ ]:
# Accuracy
print("Accuracy Trivial Model: ", accuracy_score(undersampled_java_Y_test, undersampled_Y_pred_baseline))
print("Accuracy KNN: ", accuracy_score(undersampled_java_Y_test, undersampled_Y_pred_knn))
print("Accuracy SVM: ", accuracy_score(undersampled_java_Y_test, undersampled_Y_pred_svm))
print("Accuracy LogisticRegression: ", accuracy_score(undersampled_java_Y_test, undersampled_Y_pred_logistic_regression))
print("Accuracy Random Forest: ", accuracy_score(undersampled_java_Y_test, undersampled_Y_pred_random_forest))
print("Accuracy GBM: ", accuracy_score(undersampled_java_Y_test, undersampled_Y_pred_gbm))

Accuracy Trivial Model:  0.5
Accuracy SVM:  0.6626344086021505
Accuracy LogisticRegression:  0.6536738351254481
Accuracy Random Forest:  0.6545698924731183
Accuracy KNN:  0.589605734767025
Accuracy GBM:  0.6317204301075269


In [ ]:
# F1 score
print("F1-score Trivial Model: ", f1_score(undersampled_java_Y_test, undersampled_Y_pred_baseline, pos_label = "P3"))
print("F1-score KNN: ", f1_score(undersampled_java_Y_test, undersampled_Y_pred_knn, pos_label = "P3"))
print("F1-score SVM: ", f1_score(undersampled_java_Y_test, undersampled_Y_pred_svm, pos_label = "P3"))
print("F1-score LogisticRegression: ", f1_score(undersampled_java_Y_test, undersampled_Y_pred_logistic_regression, pos_label = "P3"))
print("F1-score Random Forest: ", f1_score(undersampled_java_Y_test, undersampled_Y_pred_random_forest, pos_label = "P3"))
print("F1-score GBM: ", f1_score(undersampled_java_Y_test, undersampled_Y_pred_gbm, pos_label = "P3"))

F1-score Trivial Model:  0.6666666666666666
F1-score SVM:  0.6669615214506855
F1-score LogisticRegression:  0.6608161474330847
F1-score Random Forest:  0.6737198476512907
F1-score KNN:  0.6321285140562248
F1-score GBM:  0.6663961038961039


In [ ]:
# Cohen's Kappa
print("Cohen's kappa Trivial Model: ", cohen_kappa_score(undersampled_java_Y_test, undersampled_Y_pred_baseline))
print("Cohen's kappa KNN: ", cohen_kappa_score(undersampled_java_Y_test, undersampled_Y_pred_knn))
print("Cohen's kappa SVM: ", cohen_kappa_score(undersampled_java_Y_test, undersampled_Y_pred_svm))
print("Cohen's kappa LogisticRegression: ", cohen_kappa_score(undersampled_java_Y_test, undersampled_Y_pred_logistic_regression))
print("Cohen's kappa Random Forest: ", cohen_kappa_score(undersampled_java_Y_test, undersampled_Y_pred_random_forest))
print("Cohen's kappa GBM: ", cohen_kappa_score(undersampled_java_Y_test, undersampled_Y_pred_gbm))

Cohen's kappa Trivial Model:  0.0
Cohen's kappa SVM:  0.3252688172043011
Cohen's kappa LogisticRegression:  0.30734767025089604
Cohen's kappa Random Forest:  0.3091397849462365
Cohen's kappa KNN:  0.1792114695340502
Cohen's kappa GBM:  0.26344086021505375


In [ ]:
# ROC AUC
undersampled_label_encoder = LabelEncoder()
undersampled_encoded_java_Y_test = undersampled_label_encoder.fit_transform(undersampled_java_Y_test)
undersampled_encoded_Y_pred_baseline = undersampled_label_encoder.fit_transform(undersampled_Y_pred_baseline)
undersampled_encoded_Y_pred_svm = undersampled_label_encoder.fit_transform(undersampled_Y_pred_svm)
undersampled_encoded_Y_pred_logistic_regression = undersampled_label_encoder.fit_transform(undersampled_Y_pred_logistic_regression)
undersampled_encoded_Y_pred_random_forest = undersampled_label_encoder.fit_transform(undersampled_Y_pred_random_forest)
undersampled_encoded_Y_pred_knn = undersampled_label_encoder.fit_transform(undersampled_Y_pred_knn)
undersampled_encoded_Y_pred_gbm = undersampled_label_encoder.fit_transform(undersampled_Y_pred_gbm)
print("ROC AUC Trivial Model: ", roc_auc_score(undersampled_encoded_java_Y_test, undersampled_encoded_Y_pred_baseline))
print("ROC AUC KNN: ", roc_auc_score(undersampled_encoded_java_Y_test, undersampled_encoded_Y_pred_knn))
print("ROC AUC SVM: ", roc_auc_score(undersampled_encoded_java_Y_test, undersampled_encoded_Y_pred_svm))
print("ROC AUC LogisticRegression: ", roc_auc_score(undersampled_encoded_java_Y_test, undersampled_encoded_Y_pred_logistic_regression))
print("ROC AUC Random Forest: ", roc_auc_score(undersampled_encoded_java_Y_test, undersampled_encoded_Y_pred_random_forest))
print("ROC AUC GBM: ", roc_auc_score(undersampled_encoded_java_Y_test, undersampled_encoded_Y_pred_gbm))

ROC AUC Trivial Model:  0.5
ROC AUC SVM:  0.6626344086021505
ROC AUC LogisticRegression:  0.6536738351254481
ROC AUC Random Forest:  0.6545698924731183
ROC AUC KNN:  0.589605734767025
ROC AUC GBM:  0.6317204301075269


In [ ]:
# PRC AUC
print("PRC AUC Trivial Model: ", average_precision_score(undersampled_encoded_java_Y_test, undersampled_encoded_Y_pred_baseline))
print("PRC AUC KNN: ", average_precision_score(undersampled_encoded_java_Y_test, undersampled_encoded_Y_pred_knn))
print("PRC AUC SVM: ", average_precision_score(undersampled_encoded_java_Y_test, undersampled_encoded_Y_pred_svm))
print("PRC AUC LogisticRegression: ", average_precision_score(undersampled_encoded_java_Y_test, undersampled_encoded_Y_pred_logistic_regression))
print("PRC AUC Random Forest: ", average_precision_score(undersampled_encoded_java_Y_test, undersampled_encoded_Y_pred_random_forest))
print("PRC AUC GBM: ", average_precision_score(undersampled_encoded_java_Y_test, undersampled_encoded_Y_pred_gbm))

PRC AUC Trivial Model:  0.5
PRC AUC SVM:  0.6084728116251694
PRC AUC LogisticRegression:  0.6014908583709695
PRC AUC Random Forest:  0.6043542928879428
PRC AUC KNN:  0.5552464262141682
PRC AUC GBM:  0.5877639517345399


In [ ]:
# H-measure
print("H-measure Trivial Model: ", h_measure(undersampled_encoded_java_Y_test, undersampled_encoded_Y_pred_baseline))
print("H-measure KNN: ", h_measure(undersampled_encoded_java_Y_test, undersampled_encoded_Y_pred_knn))
print("H-measure SVM: ", h_measure(undersampled_encoded_java_Y_test, undersampled_encoded_Y_pred_svm))
print("H-measure LogisticRegression: ", h_measure(undersampled_encoded_java_Y_test, undersampled_encoded_Y_pred_logistic_regression))
print("H-measure Random Forest: ", h_measure(undersampled_encoded_java_Y_test, undersampled_encoded_Y_pred_random_forest))
print("H-measure GBM: ", h_measure(undersampled_encoded_java_Y_test, undersampled_encoded_Y_pred_gbm))

H-measure Trivial Model:  nan
H-measure SVM:  0.658193372673627
H-measure LogisticRegression:  0.6462242562929061
H-measure Random Forest:  0.6330318895763922
H-measure KNN:  0.53596757852077
H-measure GBM:  0.589


##**Step 5: Oversampling the minority class**

In [ ]:
# Calculate the ratio of majority to minority observations
ratio = len(majority) // len(minority)

In [ ]:
# We duplicate the observations in the minority class according to our ratio
duplicated_minority = pd.concat([minority] * ratio)

In [ ]:
# Combine the oversampled minority class with the majority class
oversampled_java = pd.concat([majority, duplicated_minority])

In [ ]:
oversampled_java.head()

,Priority,Text
1,P3,README: Hit count not reset (1GET20Y) JGS (6/4/01 11:43:47 AM); Set a breakpoint in InfiniteLoop with a hit count. When suspended;; right click breakpoint in Breakpoints viewer and enable. Hit cou...
2,P3,Use styled text in console (1G9S1YF) DW (2/26/01 5:56:36 PM); Could allow users to use styled text in console; with preference support. This would; allow console input to be shown in bold; etc.
3,P3,StringBuffer representation (1GE3BFA) JGS (5/21/01 2:09:10 PM); To be consistent; should we represent StringBuffer objects in the variables view like; we do Strings; with their values shown on the...
5,P3,Breakpoint in an invalid location (1G4F8P8) DS (11/14/00 2:59:11 PM); You can set a breakpoint that is in an invalid location; for example:; ; public class Inner implements Enumeration {; private ...
8,P3,Snippet evaluation should support imports (1G47213) I want to write code like this in a snippet file; ; import com.ibm.swt.widgets.*; ; Shell shell= new Shell();; shell.open();; ; ; NOTES:; EG (11...


In [ ]:
oversampled_java['Priority'].value_counts(normalize = True)

P3     0.50
else   0.50
Name: Priority, dtype: float64

In [ ]:
# Split the oversampled dataset into the training and test sets
oversampled_java_X_train, oversampled_java_X_test, oversampled_java_Y_train, oversampled_java_Y_test = train_test_split(oversampled_java["Text"],
                                                                        oversampled_java["Priority"],
                                                                        test_size = 0.2, # We still use 80% as training and 20% as test sets
                                                                        random_state = 1,
                                                                        stratify = oversampled_java["Priority"]) # We stratify to keep the balance in classes between the train and test sets
print("Train data:", oversampled_java_X_train.shape[0])
print("Test data:", oversampled_java_X_test.shape[0])

Train data: 62957
Test data: 15740


In [ ]:
# Vectorize the train and test data
oversampled_java_X_train_tf = tfidf.fit_transform(oversampled_java_X_train)
oversampled_java_X_test_tf = tfidf.transform(oversampled_java_X_test)

In [ ]:
# Fit a new Trivial classifier
oversampled_trivial = DummyClassifier(strategy = "most_frequent", random_state = 1)
oversampled_trivial.fit(oversampled_java_X_train_tf, oversampled_java_Y_train)
oversampled_Y_pred_baseline = oversampled_trivial.predict(oversampled_java_X_test_tf)

In [ ]:
# K-Nearest Neighbor
%%time
oversampled_knn = KNeighborsClassifier(n_neighbors = 5, metric = "euclidean")
oversampled_knn.fit(oversampled_java_X_train_tf, oversampled_java_Y_train)
oversampled_Y_pred_knn = oversampled_knn.predict(oversampled_java_X_test_tf)

CPU times: user 1h 48min 49s, sys: 4.98 s, total: 1h 48min 54s
Wall time: 1h 49min 3s


In [ ]:
# Support Vector Machine
%%time
oversampled_svm = SVC()
oversampled_svm.fit(oversampled_java_X_train_tf, oversampled_java_Y_train)
oversampled_Y_pred_svm = oversampled_svm.predict(oversampled_java_X_test_tf)

CPU times: user 1h 11min 28s, sys: 4.33 s, total: 1h 11min 32s
Wall time: 1h 11min 42s


In [ ]:
# Logistic Regression
%%time
oversampled_logistic_regression = LogisticRegression(n_jobs = -1)
oversampled_logistic_regression.fit(oversampled_java_X_train_tf, oversampled_java_Y_train)
oversampled_Y_pred_logistic_regression = oversampled_logistic_regression.predict(oversampled_java_X_test_tf)

CPU times: user 143 ms, sys: 192 ms, total: 334 ms
Wall time: 4.11 s


In [ ]:
# Random Forest
%%time
oversampled_random_forest = RandomForestClassifier(random_state = 1, n_jobs = -1)
oversampled_random_forest.fit(oversampled_java_X_train_tf, oversampled_java_Y_train)
oversampled_Y_pred_random_forest = oversampled_random_forest.predict(oversampled_java_X_test_tf)

CPU times: user 4min 22s, sys: 573 ms, total: 4min 23s
Wall time: 2min 24s


In [ ]:
# GradientBoostingClassifier
%%time
oversampled_gbm = GradientBoostingClassifier()
oversampled_gbm.fit(oversampled_java_X_train_tf, oversampled_java_Y_train)
oversampled_Y_pred_gbm = oversampled_gbm.predict(oversampled_java_X_test_tf)

CPU times: user 2min 47s, sys: 154 ms, total: 2min 48s
Wall time: 2min 48s


In [ ]:
# Accuracy
print("Accuracy Trivial Model: ", accuracy_score(oversampled_java_Y_test, oversampled_Y_pred_baseline))
print("Accuracy KNN: ", accuracy_score(oversampled_java_Y_test, oversampled_Y_pred_knn))
print("Accuracy SVM: ", accuracy_score(oversampled_java_Y_test, oversampled_Y_pred_svm))
print("Accuracy LogisticRegression: ", accuracy_score(oversampled_java_Y_test, oversampled_Y_pred_logistic_regression))
print("Accuracy Random Forest: ", accuracy_score(oversampled_java_Y_test, oversampled_Y_pred_random_forest))
print("Accuracy GBM: ", accuracy_score(oversampled_java_Y_test, oversampled_Y_pred_gbm))

Accuracy Trivial Model:  0.5037484116899619
Accuracy KNN:  0.9649301143583228
Accuracy SVM:  0.9801778907242694
Accuracy LogisticRegression:  0.8549555273189327
Accuracy Random Forest:  0.9864675984752224
Accuracy GBM:  0.6907878017789072


In [ ]:
# F1 score
print("F1-score Trivial Model: ", f1_score(oversampled_java_Y_test, oversampled_Y_pred_baseline, pos_label = "P3"))
print("F1-score KNN: ", f1_score(oversampled_java_Y_test, oversampled_Y_pred_knn, pos_label = "P3"))
print("F1-score SVM: ", f1_score(oversampled_java_Y_test, oversampled_Y_pred_svm, pos_label = "P3"))
print("F1-score LogisticRegression: ", f1_score(oversampled_java_Y_test, oversampled_Y_pred_logistic_regression, pos_label = "P3"))
print("F1-score Random Forest: ", f1_score(oversampled_java_Y_test, oversampled_Y_pred_random_forest, pos_label = "P3"))
print("F1-score GBM: ", f1_score(oversampled_java_Y_test, oversampled_Y_pred_gbm, pos_label = "P3"))

F1-score Trivial Model:  0.6699902826481896
F1-score KNN:  0.9644604687097605
F1-score SVM:  0.9800153727901615
F1-score LogisticRegression:  0.8498520223610655
F1-score Random Forest:  0.9863975988249569
F1-score GBM:  0.6934559425584178


In [ ]:
# Cohen's Kappa
print("Cohen's kappa Trivial Model: ", cohen_kappa_score(oversampled_java_Y_test, oversampled_Y_pred_baseline))
print("Cohen's kappa KNN: ", cohen_kappa_score(oversampled_java_Y_test, oversampled_Y_pred_knn))
print("Cohen's kappa SVM: ", cohen_kappa_score(oversampled_java_Y_test, oversampled_Y_pred_svm))
print("Cohen's kappa LogisticRegression: ", cohen_kappa_score(oversampled_java_Y_test, oversampled_Y_pred_logistic_regression))
print("Cohen's kappa Random Forest: ", cohen_kappa_score(oversampled_java_Y_test, oversampled_Y_pred_random_forest))
print("Cohen's kappa GBM: ", cohen_kappa_score(oversampled_java_Y_test, oversampled_Y_pred_gbm))

Cohen's kappa Trivial Model:  0.0
Cohen's kappa KNN:  0.9298780634947142
Cohen's kappa SVM:  0.9603628421306648
Cohen's kappa LogisticRegression:  0.7100751039833411
Cohen's kappa Random Forest:  0.9729388058728075
Cohen's kappa GBM:  0.3815296503288155


In [ ]:
# ROC AUC
oversampled_label_encoder = LabelEncoder()
oversampled_encoded_java_Y_test = oversampled_label_encoder.fit_transform(oversampled_java_Y_test)
oversampled_encoded_Y_pred_baseline = oversampled_label_encoder.fit_transform(oversampled_Y_pred_baseline)
oversampled_encoded_Y_pred_svm = oversampled_label_encoder.fit_transform(oversampled_Y_pred_svm)
oversampled_encoded_Y_pred_logistic_regression = oversampled_label_encoder.fit_transform(oversampled_Y_pred_logistic_regression)
oversampled_encoded_Y_pred_random_forest = oversampled_label_encoder.fit_transform(oversampled_Y_pred_random_forest)
oversampled_encoded_Y_pred_knn = oversampled_label_encoder.fit_transform(oversampled_Y_pred_knn)
oversampled_encoded_Y_pred_gbm = oversampled_label_encoder.fit_transform(oversampled_Y_pred_gbm)
print("ROC AUC Trivial Model: ", roc_auc_score(oversampled_encoded_java_Y_test, oversampled_encoded_Y_pred_baseline))
print("ROC AUC KNN: ", roc_auc_score(oversampled_encoded_java_Y_test, oversampled_encoded_Y_pred_knn))
print("ROC AUC SVM: ", roc_auc_score(oversampled_encoded_java_Y_test, oversampled_encoded_Y_pred_svm))
print("ROC AUC LogisticRegression: ", roc_auc_score(oversampled_encoded_java_Y_test, oversampled_encoded_Y_pred_logistic_regression))
print("ROC AUC Random Forest: ", roc_auc_score(oversampled_encoded_java_Y_test, oversampled_encoded_Y_pred_random_forest))
print("ROC AUC GBM: ", roc_auc_score(oversampled_encoded_java_Y_test, oversampled_encoded_Y_pred_gbm))

ROC AUC Trivial Model:  0.5
ROC AUC KNN:  0.9650834228932201
ROC AUC SVM:  0.9802939508312951
ROC AUC LogisticRegression:  0.855258410649023
ROC AUC Random Forest:  0.9865616251542645
ROC AUC GBM:  0.6907613723053139


In [ ]:
# PRC AUC
print("PRC AUC Trivial Model: ", average_precision_score(oversampled_encoded_java_Y_test, oversampled_encoded_Y_pred_baseline))
print("PRC AUC KNN: ", average_precision_score(oversampled_encoded_java_Y_test, oversampled_encoded_Y_pred_knn))
print("PRC AUC SVM: ", average_precision_score(oversampled_encoded_java_Y_test, oversampled_encoded_Y_pred_svm))
print("PRC AUC LogisticRegression: ", average_precision_score(oversampled_encoded_java_Y_test, oversampled_encoded_Y_pred_logistic_regression))
print("PRC AUC Random Forest: ", average_precision_score(oversampled_encoded_java_Y_test, oversampled_encoded_Y_pred_random_forest))
print("PRC AUC GBM: ", average_precision_score(oversampled_encoded_java_Y_test, oversampled_encoded_Y_pred_gbm))

PRC AUC Trivial Model:  0.4962515883100381
PRC AUC KNN:  0.9395417945136552
PRC AUC SVM:  0.963389782452304
PRC AUC LogisticRegression:  0.7920952029809273
PRC AUC Random Forest:  0.9738537496826404
PRC AUC GBM:  0.6286545460234101


In [ ]:
# H-measure
print("H-measure Trivial Model: ", h_measure(oversampled_encoded_java_Y_test, oversampled_encoded_Y_pred_baseline))
print("H-measure KNN: ", h_measure(oversampled_encoded_java_Y_test, oversampled_encoded_Y_pred_knn))
print("H-measure SVM: ", h_measure(oversampled_encoded_java_Y_test, oversampled_encoded_Y_pred_svm))
print("H-measure LogisticRegression: ", h_measure(oversampled_encoded_java_Y_test, oversampled_encoded_Y_pred_logistic_regression))
print("H-measure Random Forest: ", h_measure(oversampled_encoded_java_Y_test, oversampled_encoded_Y_pred_random_forest))
print("H-measure GBM: ", h_measure(oversampled_encoded_java_Y_test, oversampled_encoded_Y_pred_gbm))

H-measure Trivial Model:  nan
H-measure KNN:  0.9653875094055681
H-measure SVM:  0.98033778674061
H-measure LogisticRegression:  0.8597235023041475
H-measure Random Forest:  0.9865368813602174
H-measure GBM:  0.6880728065115683


##**Step 6: SMOTE**

In [ ]:
# We import the SMOTE class
from imblearn.over_sampling import SMOTE

In [ ]:
# Convert 'P3' to 1 and 'else' to 0
java['Priority'] = java['Priority'].str.replace('P3', '1')
java['Priority'] = java['Priority'].str.replace('else', '0')

In [ ]:
# Ensure that these new binary outcomes are of type integer
java['Priority'] = java['Priority'].astype(int)

In [ ]:
# We define X and Y from the original java dataset
java_X = java['Text']
java_Y = java['Priority']

In [ ]:
# Apply TF-IDF Vectorization on X
java_X_tf = tfidf.fit_transform(java_X)

In [ ]:
# Instantiate SMOTE
smote = SMOTE(random_state = 1)

In [ ]:
# Apply SMOTE algorithm on the X and Y
%%time
smote_java_X_tf, smote_java_Y = smote.fit_resample(java_X_tf, java_Y)

CPU times: user 4min 14s, sys: 377 ms, total: 4min 15s
Wall time: 4min 22s


In [ ]:
# Divide the SMOTE dataset into train and test sets
smote_java_X_train, smote_java_X_test, smote_java_Y_train, smote_java_Y_test = train_test_split(smote_java_X_tf,
                                                                        smote_java_Y,
                                                                        test_size = 0.2, # We use 80% as training and 20% as test sets
                                                                        random_state = 1,
                                                                        stratify = smote_java_Y) # We stratify to keep the balanced class distribution in the training and test samples
print("Train data:", smote_java_X_train.shape[0])
print("Test data:", smote_java_X_test.shape[0])

Train data: 63430
Test data: 15858


In [ ]:
# Trivial classifier
trivial_smote = DummyClassifier(strategy = "most_frequent", random_state = 1)
trivial_smote.fit(smote_java_X_train, smote_java_Y_train)
smote_Y_pred_baseline = trivial_smote.predict(smote_java_X_test)

In [ ]:
# K-Nearest Neighbor
%%time
knn_smote = KNeighborsClassifier(n_neighbors = 5, metric = "euclidean")
knn_smote.fit(smote_java_X_train, smote_java_Y_train)
smote_Y_pred_knn = knn_smote.predict(smote_java_X_test)

CPU times: user 4h 6min 20s, sys: 16.9 s, total: 4h 6min 36s
Wall time: 4h 8min 18s


In [ ]:
# As we are not able to run the K-Nearest Neighbor due to not enough computing power, we train the KNN on only 30% of the training data, sampled randomly
sample_smote_java_X_train, _, sample_smote_java_Y_train, _ = train_test_split(smote_java_X_train,
                                                                        smote_java_Y_train,
                                                                        test_size = 0.3,
                                                                        random_state = 1,
                                                                        stratify = smote_java_Y_train)

In [ ]:
# K-Nearest Neighbor on sample
%%time
knn_smote = KNeighborsClassifier(n_neighbors = 5, metric = "euclidean")
knn_smote.fit(sample_smote_java_X_train, sample_smote_java_Y_train)
smote_Y_pred_knn = knn_smote.predict(smote_java_X_test)

CPU times: user 1h 59min 43s, sys: 6.17 s, total: 1h 59min 50s
Wall time: 2h 25s


In [ ]:
# Support Vector Machine
%%time
svm_smote = SVC()
svm_smote.fit(smote_java_X_train, smote_java_Y_train)
smote_Y_pred_svm = svm_smote.predict(smote_java_X_test)

CPU times: user 1h 42min 39s, sys: 9.19 s, total: 1h 42min 48s
Wall time: 1h 43min 41s


In [ ]:
# Logistic Regression
%%time
logistic_regression_smote = LogisticRegression(n_jobs = -1)
logistic_regression_smote.fit(smote_java_X_train, smote_java_Y_train)
smote_Y_pred_logistic_regression = logistic_regression_smote.predict(smote_java_X_test)

CPU times: user 241 ms, sys: 372 ms, total: 613 ms
Wall time: 7.94 s


In [ ]:
# Random Forest
%%time
random_forest_smote = RandomForestClassifier(random_state = 1, n_jobs = -1)
random_forest_smote.fit(smote_java_X_train, smote_java_Y_train)
smote_Y_pred_random_forest = random_forest_smote.predict(smote_java_X_test)

In [ ]:
# Gradient Boosting Classifier
%%time
gbm_smote = GradientBoostingClassifier()
gbm_smote.fit(smote_java_X_train, smote_java_Y_train)
smote_Y_pred_gbm = gbm_smote.predict(smote_java_X_test)

In [ ]:
# Accuracy
print("Accuracy Trivial Model: ", accuracy_score(smote_java_Y_test, smote_Y_pred_baseline))
print("Accuracy KNN: ", accuracy_score(smote_java_Y_test, smote_Y_pred_knn))
print("Accuracy SVM: ", accuracy_score(smote_java_Y_test, smote_Y_pred_svm))
print("Accuracy LogisticRegression: ", accuracy_score(smote_java_Y_test, smote_Y_pred_logistic_regression))
print("Accuracy Random Forest: ", accuracy_score(smote_java_Y_test, smote_Y_pred_random_forest))
print("Accuracy GBM: ", accuracy_score(smote_java_Y_test, smote_Y_pred_gbm))

In [ ]:
# F1 score
print("F1-score Trivial Model: ", f1_score(smote_java_Y_test, smote_Y_pred_baseline, pos_label = "P3"))
print("F1-score KNN: ", f1_score(smote_java_Y_test, smote_Y_pred_knn, pos_label = "P3"))
print("F1-score SVM: ", f1_score(smote_java_Y_test, smote_Y_pred_svm, pos_label = "P3"))
print("F1-score LogisticRegression: ", f1_score(smote_java_Y_test, smote_Y_pred_logistic_regression, pos_label = "P3"))
print("F1-score Random Forest: ", f1_score(smote_java_Y_test, smote_Y_pred_random_forest, pos_label = "P3"))
print("F1-score GBM: ", f1_score(smote_java_Y_test, smote_Y_pred_gbm, pos_label = "P3"))

In [ ]:
# Cohen's Kappa
print("Cohen's kappa Trivial Model: ", cohen_kappa_score(smote_java_Y_test, smote_Y_pred_baseline))
print("Cohen's kappa KNN: ", cohen_kappa_score(smote_java_Y_test, smote_Y_pred_knn))
print("Cohen's kappa SVM: ", cohen_kappa_score(smote_java_Y_test, smote_Y_pred_svm))
print("Cohen's kappa LogisticRegression: ", cohen_kappa_score(smote_java_Y_test, smote_Y_pred_logistic_regression))
print("Cohen's kappa Random Forest: ", cohen_kappa_score(smote_java_Y_test, smote_Y_pred_random_forest))
print("Cohen's kappa GBM: ", cohen_kappa_score(smote_java_Y_test, smote_Y_pred_gbm))

In [ ]:
# ROC AUC
smote_label_encoder = LabelEncoder()
smote_encoded_java_Y_test = smote_label_encoder.fit_transform(smote_java_Y_test)
smote_encoded_Y_pred_baseline = smote_label_encoder.fit_transform(smote_Y_pred_baseline)
smote_encoded_Y_pred_knn = smote_label_encoder.fit_transform(smote_Y_pred_knn)
smote_encoded_Y_pred_svm = smote_label_encoder.fit_transform(smote_Y_pred_svm)
smote_encoded_Y_pred_logistic_regression = smote_label_encoder.fit_transform(smote_Y_pred_logistic_regression)
smote_encoded_Y_pred_random_forest = smote_label_encoder.fit_transform(smote_Y_pred_random_forest)
smote_encoded_Y_pred_gbm = smote_label_encoder.fit_transform(smote_Y_pred_gbm)
print("ROC AUC Trivial Model: ", roc_auc_score(smote_encoded_java_Y_test, smote_encoded_Y_pred_baseline))
print("ROC AUC KNN: ", roc_auc_score(smote_encoded_java_Y_test, smote_encoded_Y_pred_knn))
print("ROC AUC SVM: ", roc_auc_score(smote_encoded_java_Y_test, smote_encoded_Y_pred_svm))
print("ROC AUC LogisticRegression: ", roc_auc_score(smote_encoded_java_Y_test, smote_encoded_Y_pred_logistic_regression))
print("ROC AUC Random Forest: ", roc_auc_score(smote_encoded_java_Y_test, smote_encoded_Y_pred_random_forest))
print("ROC AUC GBM: ", roc_auc_score(smote_encoded_java_Y_test, smote_encoded_Y_pred_gbm))

In [ ]:
# PRC AUC
print("PRC AUC Trivial Model: ", average_precision_score(smote_encoded_java_Y_test, smote_encoded_Y_pred_baseline))
print("PRC AUC KNN: ", average_precision_score(smote_encoded_java_Y_test, smote_encoded_Y_pred_knn))
print("PRC AUC SVM: ", average_precision_score(smote_encoded_java_Y_test, smote_encoded_Y_pred_svm))
print("PRC AUC LogisticRegression: ", average_precision_score(smote_encoded_java_Y_test, smote_encoded_Y_pred_logistic_regression))
print("PRC AUC Random Forest: ", average_precision_score(smote_encoded_java_Y_test, smote_encoded_Y_pred_random_forest))
print("PRC AUC GBM: ", average_precision_score(smote_encoded_java_Y_test, smote_encoded_Y_pred_gbm))

In [ ]:
# H-measure
print("H-measure Trivial Model: ", h_measure(smote_encoded_java_Y_test, smote_encoded_Y_pred_baseline))
print("H-measure KNN: ", h_measure(smote_encoded_java_Y_test, smote_encoded_Y_pred_knn))
print("H-measure SVM: ", h_measure(smote_encoded_java_Y_test, smote_encoded_Y_pred_svm))
print("H-measure LogisticRegression: ", h_measure(smote_encoded_java_Y_test, smote_encoded_Y_pred_logistic_regression))
print("H-measure Random Forest: ", h_measure(smote_encoded_java_Y_test, smote_encoded_Y_pred_random_forest))
print("H-measure GBM: ", h_measure(smote_encoded_java_Y_test, smote_encoded_Y_pred_gbm))